In [ ]:
!pip install PyTDC smiles-encoder scipy

In [ ]:
!pip install scikit-learn

In [ ]:
from tdc.single_pred import Tox
from sklearn import svm

data = Tox(name = 'hERG')
df = data.get_data()
df


#https://pypi.org/project/smiles-encoder/ smile encoder
#workflow
# encode SMILES strings --> binary classification
#timeline
#gather packages needed
#have funtional model
# refine model parameters
# create

In [ ]:
split = data.get_split()
smiles_list_train = split['train']['Drug'].tolist()
Y_train = split['train']['Y']
Y_train

In [ ]:
from smiles_encoder import SmilesEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np


#encode into vectors using one hot encoding 
encoder = SmilesEncoder(smiles_list_train)
X_train = encoder.encode_many(smiles_list_train)
encoder_valid = SmilesEncoder(split['valid']['Drug'].tolist())
X_valid = encoder_valid.encode_many(split['valid']['Drug'].tolist())
Y_train = split['train']['Y']


# Pad sequences to a fixed length
# Step 1: Determine maximum sizes
max_outer = max(max(len(sample) for sample in X_train), max(len(sample) for sample in X_valid))  # Outer dimension
max_inner = max(
    max(len(inner) for sample in X_train for inner in sample),
    max(len(inner) for sample in X_valid for inner in sample),
)  # Inner dimension

# Step 2: Function to pad both outer and inner dimensions
def pad_samples(data, max_outer, max_inner):
    padded = []
    for sample in data:
        # Pad inner lists to max_inner
        padded_sample = [inner + [0] * (max_inner - len(inner)) for inner in sample]
        # Pad the outer list to max_outer with zero-filled inner lists
        padded_sample += [[0] * max_inner] * (max_outer - len(sample))
        padded.append(padded_sample)
    return np.array(padded)
# Apply padding to both datasets
X_train_padded = pad_samples(X_train, max_outer, max_inner)
X_val_padded = pad_samples(X_valid, max_outer, max_inner)


#flatten 
X_train_final = [np.array(sample).flatten() for sample in X_train_padded]

#train SVM
svm = SVC()
param_grid = {'C': [0.1, 1, 10], 'gamma': [1, 0.1, 0.01], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
grid = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='accuracy', cv=5)
grid.fit(X_train_final, Y_train)

# Evaluate
print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)



In [ ]:

X_valid_final = [np.array(sample).flatten() for sample in X_val_padded]
Y_valid = split['valid']['Y']

# Evaluate on validation data
best_model = grid.best_estimator_
Y_pred_valid = best_model.predict(X_valid_final)

# Validation metrics
print("Validation accuracy:", accuracy_score(Y_valid, Y_pred_valid))
print("Classification report:\n", classification_report(Y_valid, Y_pred_valid))